In [1]:
import sys
from importlib import reload
import sympy
from itertools import product
from collections import defaultdict

In [2]:
sys.path.insert(0, '../')

In [363]:
import multivector as mv
import examples as ex
import embedding as emb

```
t*(x0,x1,x2,y0,y1,y2) = (t^2*x0, t*x1, t*x2, t-2*y0, t^-1*y1, t^-1*y2)

invariants:
x0*y0
x1*y1
x2*y2
x1*y2
x2*y1
x0*y1^2
x0*y1*y2
x0*y2^2
y0*x1^2
y0*x1*x2
y0*x2^2
```

In [173]:
#a = sympy.symbols(['a%d' %x for x in range(11)])
q = sympy.symbols(tuple('q%d' %x for x in range(5)))
a = sympy.symbols(tuple('a%d' %x for x in range(11)))
b = sympy.symbols(tuple('b%d' %x for x in range(11)))
emb_coords = q + a + b


x = sympy.symbols('x0 x1 x2')
y = sympy.symbols('y0 y1 y2')
stdbrac = mv.Mv(x+y, {(x[0],y[0]):1, (x[1],y[1]):1}).brac

In [281]:
rmap = {
    q[0]: x[0]*y[0], 
    q[1]: x[1]*y[1],
    q[2]: x[2]*y[2],
    q[3]: x[1]*y[2],
    q[4]: x[2]*y[1],
    a[0]: x[0]*(y[1]**2),
    a[1]: x[0]*(y[1]*y[2]),
    a[2]: x[0]*(y[2]**2),
    b[0]: y[0]*(x[1]**2),
    b[1]: y[0]*(x[1]*x[2]),
    b[2]: y[0]*(x[2]**2)
}

embcoords, invariants = list(zip(*rmap.items()))

In [282]:
gbK = sympy.polys.groebner([ai-xi for (ai,xi) in rmap.items()])
K = [g for g in gbK if not (set(g.as_poly().gens) - set(embcoords))]

In [283]:
KHom = {sympy.symbols('K%d' %x):y for (x,y) in enumerate(K)}

kerCoords, kerPolys = list(zip(*KHom.items()))

In [284]:
P = {}
for k1, k2 in product(rmap.keys(), rmap.keys()):
    if (k2, k1) not in P.keys():
        P[(k1, k2)] = brac(rmap[k1],rmap[k2])

In [288]:
lift = emb.lift(rmap, embcoords, x+y)

In [289]:
Pi = mv.Mv(embcoords, P).mapCoeffs(lambda x:lift(x))

In [290]:
Pi.mapCoeffs(lambda x:x.simplify())

a0*da0*dq0 - 2*a0*da0*dq1 - 2*a1*da0*dq3 - a1*dq0*da1 + a1*dq1*da1 - a2*dq0*da2 + a2*dq3*da1 + 2*b0*db0*dq1 + b0*dq0*db0 + b1*db1*dq1 + b1*dq0*db1 - 2*b1*dq4*db0 + b2*dq0*db2 - b2*dq4*db1 + q1*(-4*q0 + q1)*da0*db0 - q2**2*db2*da2 - q2*q3*db1*da2 - q2*q4*db2*da1 + q2*(q0 - q1)*db1*da1 - q2*dq4*dq3 - q3**2*db0*da2 + q3*(2*q0 - q1)*db0*da1 + q3*dq3*dq1 + q4**2*da0*db2 + q4*(-2*q0 + q1)*da0*db1 - q4*dq4*dq1

In [291]:
PiDegs = Pi.deg()

In [362]:
mv.sBr(Pi,Pi).sort().mapCoeffs(lambda x:x.simplify()/2) == 0

False

In [297]:
P = {}
coeffs = []
A = len(embcoords)
index_cnt = 0
for i0 in range(A):
    for i1 in range(i0):
        coeffs_tmp = sympy.symbols(['c_%d' %x for x in range(index_cnt, index_cnt + len(KHom.values()))])
        coeffs.extend(coeffs_tmp)
        index_cnt += len(KHom.values())
        cterms = sum(x*y for (x,y) in zip(coeffs_tmp, KHom.values()))
        P[(embcoords[i0], embcoords[i1])] = cterms


In [298]:
alpha = mv.Mv(embcoords, P).sort()

In [303]:
alpha_degs, Pi_degs = alpha.deg(), Pi.deg()

In [306]:
alpha_degs.keys(), Pi_degs.keys()

(dict_keys([2, 3]), dict_keys([1, 2]))

In [307]:
Jac2 = mv.sBr(Pi_degs[1],alpha_degs[2]).sort()

In [312]:
system = []
for f in Jac2._P.values():
    system.extend(f.expand().as_poly(embcoords).coeffs())

In [330]:
coeff_in_system = set()
for cond in syst:
    if cond != 0:
        coeff_in_system.update(set(cond.as_poly().gens))